In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree,ensemble,metrics

from rule import Rule
from rule_extraction import rule_extract,draw_tree,rules_vote

import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from deap import creator, base, tools, algorithms
import sys
from datetime import datetime

C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))


def getFitness(individual, X, y):
    """
    Feature subset fitness function
    """

    if(individual.count(0) != len(individual)):
        # get index with value 0
        cols = [index for index in range(
            len(individual)) if individual[index] == 0]

        # get features subset
        X_parsed = X.drop(X.columns[cols], axis=1)
        X_subset = pd.get_dummies(X_parsed)

        # apply classification algorithm
        clf = LogisticRegression()

        return (avg(cross_val_score(clf, X_subset, y, cv=5)),)
    else:
        return(0,)


def geneticAlgorithm(X, y, n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
    # create individual
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat,
                     creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.individual)
    toolbox.register("evaluate", getFitness, X=X, y=y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    return hof


def bestIndividual(hof, X, y):
    """
    Get the best individual
    """
    maxAccurcy = 0.0
    for individual in hof:
    	print(individual.fitness.values)
    	print(maxAccurcy)
    	if(individual.fitness.values[0] > maxAccurcy):
            maxAccurcy = individual.fitness.values
            _individual = individual

    _individualHeader = [list(X)[i] for i in range(
        len(_individual)) if _individual[i] == 1]
    return _individual.fitness.values, _individual, _individualHeader


# def getArguments():
#     """
#     Get argumments from command-line
#     If pass only dataframe path, pop and gen will be default
#     """
#     dfPath = sys.argv[1]
#     if(len(sys.argv) == 4):
#         pop = int(sys.argv[2])
#         gen = int(sys.argv[3])
#     else:
#         pop = 10
#         gen = 2
#     return dfPath, pop, gen


def gafeature():
    # get dataframe path, population number and generation number from command-line argument
    n_pop = 20
    n_gen = 6
    # read dataframe from csv
    df = pd.read_csv('diabetes.csv')

    # encode labels column to numbers
    le = LabelEncoder()
    le.fit(df.iloc[:, -1])
    y = le.transform(df.iloc[:, -1])
    X = df.iloc[:, :-1]

    # get accuracy with all features
    individual = [1 for i in range(len(X.columns))]
    print("Accuracy with all features: \t" + str(getFitness(individual, X, y)) + "\n")

    # apply genetic algorithm
    hof = geneticAlgorithm(X, y, n_pop, n_gen)

    # select the best individual
    accuracy, individual, header = bestIndividual(hof, X, y)
    print('Best Accuracy: \t' + str(accuracy))
    print('Number of Features in Subset: \t' + str(individual.count(1)))
    print('Individual: \t\t' + str(individual))
    print('Feature Subset\t: ' + str(header))

    return header

In [3]:
gafeature()

C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\lin

Accuracy with all features: 	(0.39230861637459163,)



C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarn

C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anac

C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\lin

...

C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\lin

C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarn

C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarn

6  	11    	0.402652	0.289584	0.4205  
(0.4204996672071519,)
0.0
(0.41805058561274977,)
(0.4204996672071519,)
(0.41564177481333553,)
(0.4204996672071519,)
(0.41440008603572265,)
(0.4204996672071519,)
(0.4065303864566642,)
(0.4204996672071519,)
(0.4062972089649586,)
(0.4204996672071519,)
(0.406133369722024,)
(0.4204996672071519,)
(0.405132312317118,)
(0.4204996672071519,)
(0.40017604897025116,)
(0.4204996672071519,)
(0.3975754666808022,)
(0.4204996672071519,)
(0.39468530142652347,)
(0.4204996672071519,)
(0.3933978441788013,)
(0.4204996672071519,)
(0.39230861637459163,)
(0.4204996672071519,)
(0.3920781388855861,)
(0.4204996672071519,)
(0.39181750287585604,)
(0.4204996672071519,)
(0.3913200231330932,)
(0.4204996672071519,)
(0.390947633337987,)
(0.4204996672071519,)
(0.39087732885058885,)
(0.4204996672071519,)
(0.39010138710875936,)
(0.4204996672071519,)
(0.3883680100036466,)
(0.4204996672071519,)
(0.3871966231831282,)
(0.4204996672071519,)
(0.38576686137600796,)
(0.4204996672071519,)
(0.38

['Pregnancies',
 'Gender',
 'BloodPressure',
 'SkinThickness',
 'DiabetesPedigreeFunction',
 'Age',
 'Exercise']

In [4]:
# fetch dataset
data = pd.read_csv('diabetes.csv', usecols = ['Pregnancies', 'Gender', 'BloodPressure', 'Insulin', 'DiabetesPedigreeFunction', 'Age','Outcome'])

# drop NA records since most Tree algorithm cannot handle
data.dropna(inplace=True)

# split training/test sets
X_train, X_test, y_train, y_test = train_test_split(data[['Pregnancies', 'Gender', 'BloodPressure', 'Insulin', 'DiabetesPedigreeFunction', 'Age','Outcome']], 
                                                    data.Outcome, test_size=0.2, random_state=0)

# dataset shape
print(X_train.shape, X_test.shape)
print(X_train.head(5))

(614, 7) (154, 7)
     Pregnancies  Gender  BloodPressure  Insulin  DiabetesPedigreeFunction  \
603            0       0             78      126                     0.692   
118            4       1             60        0                     0.443   
247            0       1             90      680                     0.427   
157            1       1             56      135                     0.833   
468            0       0              0        0                     0.183   

     Age  Outcome  
603   54        1  
118   22        0  
247   23        0  
157   23        0  
468   38        1  


In [5]:
X_train_proceeded = X_train[['Pregnancies', 'Gender', 'BloodPressure', 'Insulin', 'DiabetesPedigreeFunction', 'Age']]
X_test_proceeded = X_test[['Pregnancies', 'Gender', 'BloodPressure', 'Insulin', 'DiabetesPedigreeFunction', 'Age']]
print(X_train_proceeded.head())

     Pregnancies  Gender  BloodPressure  Insulin  DiabetesPedigreeFunction  \
603            0       0             78      126                     0.692   
118            4       1             60        0                     0.443   
247            0       1             90      680                     0.427   
157            1       1             56      135                     0.833   
468            0       0              0        0                     0.183   

     Age  
603   54  
118   22  
247   23  
157   23  
468   38  


In [6]:
model_RF_clf = ensemble.RandomForestClassifier(max_depth=4,n_estimators=3,random_state=9)
model_RF_clf.fit(X_train_proceeded,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=4, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=3,
                       n_jobs=None, oob_score=False, random_state=9, verbose=0,
                       warm_start=False)

In [7]:
# model performance on test set
y_pred_test = model_RF_clf.predict(X_test_proceeded)
print(metrics.confusion_matrix(y_test,y_pred_test))

[[98  9]
 [35 12]]


In [8]:
start_time = datetime.now()
rules,_ = rule_extract(model=model_RF_clf,feature_names=X_train_proceeded.columns)
for i in rules:
    print(i)
end_time = datetime.now()

Age <= 27.5 and DiabetesPedigreeFunction <= 1.2545000314712524 and BloodPressure <= 84.5 and DiabetesPedigreeFunction > 0.19849999994039536
Age > 27.5 and Age <= 63.5 and Pregnancies > 2.5 and BloodPressure > 91.0
Age <= 27.5 and DiabetesPedigreeFunction <= 1.2545000314712524 and BloodPressure > 84.5 and BloodPressure > 97.0
Age <= 27.5 and DiabetesPedigreeFunction > 1.2545000314712524 and Pregnancies > 2.0
Age > 27.5 and Age <= 63.5 and Pregnancies <= 2.5 and BloodPressure <= 101.0
Age > 27.5 and Age <= 63.5 and Pregnancies > 2.5 and BloodPressure <= 91.0
Age <= 27.5 and DiabetesPedigreeFunction <= 1.2545000314712524 and BloodPressure > 84.5 and BloodPressure <= 97.0
Age > 27.5 and Age > 63.5 and Age > 67.5
Age > 27.5 and Age > 63.5 and Age <= 67.5 and BloodPressure <= 90.0
Age <= 27.5 and DiabetesPedigreeFunction > 1.2545000314712524 and Pregnancies <= 2.0 and BloodPressure <= 65.0
Age > 27.5 and Age <= 63.5 and Pregnancies <= 2.5 and BloodPressure > 101.0
Age > 27.5 and Age > 63.5 a

In [10]:
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:00.006981
